In [4]:
import numpy as np
import networkx as nx
import folium
import osmnx as ox
from geopy.distance import geodesic
from collections import defaultdict

# Constants
TRAVEL_TIME_PER_KM = 5  # minutes per km
DELIVERY_TIME_PER_SHIPMENT = 10  # minutes
PRIORITY_VEHICLES = ["3W", "4W-EV"]  # Priority vehicle types

# Vehicle Information
vehicles = {
    "3W": {"capacity": 5, "max_distance": 15, "count": 50},
    "4W-EV": {"capacity": 8, "max_distance": 20, "count": 25},
    "4W": {"capacity": 25, "max_distance": float('inf'), "count": float('inf')},
}

# Get warehouse location from user
store_lat = float(input("Enter Warehouse Latitude: "))
store_lng = float(input("Enter Warehouse Longitude: "))
store_location = (store_lat, store_lng)

# Load Road Network from OpenStreetMap
place_name = "Ahmedabad, India"
graph = ox.graph_from_place(place_name, network_type="drive")
print("Road network loaded successfully!")

def compute_distance(loc1, loc2):
    """Calculate geodesic distance (great-circle distance) between two points."""
    return geodesic(loc1, loc2).km

def get_shipments_from_user():
    """Get shipment details from user input"""
    num_shipments = int(input("Enter the number of shipments: "))
    shipments = []

    for i in range(num_shipments):
        shipment_id = input(f"Enter Shipment ID for shipment {i + 1}: ")
        source_lat = float(input(f"Enter source latitude for shipment {i + 1}: "))
        source_lng = float(input(f"Enter source longitude for shipment {i + 1}: "))
        dest_lat = float(input(f"Enter destination latitude for shipment {i + 1}: "))
        dest_lng = float(input(f"Enter destination longitude for shipment {i + 1}: "))
        weight = float(input(f"Enter shipment weight (kg) for shipment {i + 1}: "))

        shipments.append({
            "id": shipment_id,
            "source": (source_lat, source_lng),
            "destination": (dest_lat, dest_lng),
            "weight": weight
        })
    return shipments

def nearest_road_node(lat, lng):
    """Find the nearest road node in the OSM graph."""
    return ox.distance.nearest_nodes(graph, lng, lat)

def find_shortest_route(shipments):
    """Find optimized route using real roads."""
    pickup_points = [s["source"] for s in shipments]
    delivery_points = [s["destination"] for s in shipments]
    all_points = [store_location] + pickup_points + delivery_points

    # Convert locations to nearest road nodes
    nodes = [nearest_road_node(lat, lng) for lat, lng in all_points]

    # Compute shortest path using Dijkstra
    route_nodes = []
    for i in range(len(nodes) - 1):
        path = nx.shortest_path(graph, source=nodes[i], target=nodes[i + 1], weight="length", method="dijkstra")
        route_nodes.extend(path[:-1])  # Avoid duplicate nodes
    route_nodes.append(nodes[-1])

    # Convert route nodes back to lat/lng
    route_coords = [(graph.nodes[n]["y"], graph.nodes[n]["x"]) for n in route_nodes]
    return route_coords

def plot_route_map(route, trip_details):
    """Plot the optimized delivery route on an interactive map."""
    center_lat = sum(lat for lat, _ in route) / len(route)
    center_lng = sum(lng for _, lng in route) / len(route)
    
    route_map = folium.Map(location=(center_lat, center_lng), zoom_start=12)

    # Add warehouse marker
    folium.Marker(
        route[0], popup='Warehouse', icon=folium.Icon(color='red', icon='home')
    ).add_to(route_map)

    # Plot route segments
    for i in range(len(route) - 1):
        points = [route[i], route[i + 1]]
        folium.PolyLine(points, weight=2, color='blue', opacity=0.8).add_to(route_map)

    # Add delivery points
    for i, point in enumerate(route[1:-1], 1):
        folium.Marker(point, popup=f'Stop {i}', icon=folium.Icon(color='blue')).add_to(route_map)

    # Save the map
    map_filename = f"Trip_{trip_details['Trip_ID']}_Route_Map.html"
    route_map.save(map_filename)
    print(f"Route map saved as {map_filename}")

def plan_trips(shipments):
    """Plan and optimize delivery trips."""
    trip_id = 1
    trips = []
    
    optimized_route = find_shortest_route(shipments)
    total_distance = sum(compute_distance(optimized_route[i], optimized_route[i + 1]) for i in range(len(optimized_route) - 1))
    total_weight = sum(s["weight"] for s in shipments)
    
    for vehicle_type, specs in vehicles.items():
        if total_weight <= specs["capacity"] and total_distance <= specs["max_distance"] and specs["count"] > 0:
            trip_time = (total_distance * TRAVEL_TIME_PER_KM + len(shipments) * DELIVERY_TIME_PER_SHIPMENT)
            capacity_utilization = (total_weight / specs["capacity"]) * 100
            on_time_delivery = total_distance / specs["max_distance"] <= 0.9
            
            trip_details = {
                "Trip_ID": f"T{trip_id}",
                "Sequence": [s["id"] for s in shipments],
                "Vehicle_Type": vehicle_type,
                "MST_DIST": total_distance,
                "TRIP_TIME": trip_time,
                "CAPACITY_UTI": capacity_utilization,
                "ON_TIME_DELIVERY": on_time_delivery,
            }
            
            print(trip_details)
            trips.append(trip_details)
            vehicles[vehicle_type]["count"] -= 1
            trip_id += 1

            plot_route_map(optimized_route, trip_details)
            break

    return trips

def main():
    """Main execution function"""
    print("Welcome to the Delivery Route Optimization System")
    
    # Get shipment data
    shipments = get_shipments_from_user()
    
    # Process and optimize routes
    print("\nProcessing and optimizing routes...")
    trips = plan_trips(shipments)

    # Display results
    print("\nOptimized Delivery Trips:")
    for trip in trips:
        print("\nTrip Details:")
        for key, value in trip.items():
            print(f"{key}: {value}")

if __name__ == "__main__":
    main()


Road network loaded successfully!
Welcome to the Delivery Route Optimization System

Processing and optimizing routes...
{'Trip_ID': 'T1', 'Sequence': ['S1', 'S2', 'S3'], 'Vehicle_Type': '4W', 'MST_DIST': 22.43813686253217, 'TRIP_TIME': 142.19068431266084, 'CAPACITY_UTI': 68.0, 'ON_TIME_DELIVERY': True}
Route map saved as Trip_T1_Route_Map.html

Optimized Delivery Trips:

Trip Details:
Trip_ID: T1
Sequence: ['S1', 'S2', 'S3']
Vehicle_Type: 4W
MST_DIST: 22.43813686253217
TRIP_TIME: 142.19068431266084
CAPACITY_UTI: 68.0
ON_TIME_DELIVERY: True


In [ ]:
import numpy as np
import networkx as nx
import folium
import osmnx as ox
from geopy.distance import geodesic

# Constants
TRAVEL_TIME_PER_KM = 5  # minutes per km
DELIVERY_TIME_PER_SHIPMENT = 10  # minutes
PRIORITY_VEHICLES = ["3W", "4W-EV"]  # Priority vehicle types

# Vehicle Information
vehicles = {
    "3W": {"capacity": 5, "max_distance": 15, "count": 50},
    "4W-EV": {"capacity": 8, "max_distance": 20, "count": 25},
    "4W": {"capacity": 25, "max_distance": float('inf'), "count": float('inf')},
}

# Get warehouse location from user
store_lat = float(input("Enter Warehouse Latitude: "))
store_lng = float(input("Enter Warehouse Longitude: "))
store_location = (store_lat, store_lng)

# Load Road Network from OpenStreetMap
place_name = "Ahmedabad, India"
graph = ox.graph_from_place(place_name, network_type="drive")
# graph = ox.simplify_graph(graph)  # Simplify graph to reduce unnecessary nodes
print("Road network loaded successfully!")

def compute_distance(loc1, loc2):
    """Calculate geodesic distance (great-circle distance) between two points."""
    return geodesic(loc1, loc2).km

def get_shipments_from_user():
    """Get shipment details from user input"""
    num_shipments = int(input("Enter the number of shipments: "))
    shipments = []

    for i in range(num_shipments):
        shipment_id = input(f"Enter Shipment ID for shipment {i + 1}: ")
        source_lat = float(input(f"Enter source latitude for shipment {i + 1}: "))
        source_lng = float(input(f"Enter source longitude for shipment {i + 1}: "))
        dest_lat = float(input(f"Enter destination latitude for shipment {i + 1}: "))
        dest_lng = float(input(f"Enter destination longitude for shipment {i + 1}: "))
        weight = float(input(f"Enter shipment weight (kg) for shipment {i + 1}: "))

        shipments.append({
            "id": shipment_id,
            "source": (source_lat, source_lng),
            "destination": (dest_lat, dest_lng),
            "weight": weight
        })
    return shipments

def nearest_road_node(lat, lng):
    """Find the nearest road node in the OSM graph."""
    return ox.distance.nearest_nodes(graph, lng, lat)

def find_shortest_route(shipments):
    """Find optimized route using only warehouse, pickup, and delivery points."""
    pickup_points = [s["source"] for s in shipments]
    delivery_points = [s["destination"] for s in shipments]
    all_points = [store_location] + pickup_points + delivery_points

    # Convert locations to nearest road nodes
    nodes = [nearest_road_node(lat, lng) for lat, lng in all_points]

    # Find shortest path between key locations (skip intermediate nodes)
    route_coords = [all_points[0]]  # Start with the warehouse
    for i in range(1, len(all_points)):
        route_coords.append(all_points[i])  # Directly add shipment points

    return route_coords  # Only the main points, not road network nodes



def plot_route_map(route, trip_details):
    """Plot the optimized delivery route on an interactive map."""
    center_lat = sum(lat for lat, _ in route) / len(route)
    center_lng = sum(lng for _, lng in route) / len(route)
    
    route_map = folium.Map(location=(center_lat, center_lng), zoom_start=12)

    # Add warehouse marker
    folium.Marker(
        route[0], popup='Warehouse', icon=folium.Icon(color='red', icon='home')
    ).add_to(route_map)

    # Plot route segments
    folium.PolyLine(route, weight=3, color='blue', opacity=0.8).add_to(route_map)

    # Add delivery points
    for i, point in enumerate(route[1:-1], 1):
        folium.Marker(point, popup=f'Stop {i}', icon=folium.Icon(color='blue')).add_to(route_map)

    # Save the map
    map_filename = f"Trip_{trip_details['Trip_ID']}_Route_Map.html"
    route_map.save(map_filename)
    print(f"Route map saved as {map_filename}")

def plan_trips(shipments):
    """Plan and optimize delivery trips."""
    trip_id = 1
    trips = []
    
    optimized_route = find_shortest_route(shipments)
    total_distance = sum(compute_distance(optimized_route[i], optimized_route[i + 1]) for i in range(len(optimized_route) - 1))
    total_weight = sum(s["weight"] for s in shipments)
    
    for vehicle_type, specs in vehicles.items():
        if total_weight <= specs["capacity"] and total_distance <= specs["max_distance"] and specs["count"] > 0:
            trip_time = (total_distance * TRAVEL_TIME_PER_KM + len(shipments) * DELIVERY_TIME_PER_SHIPMENT)
            capacity_utilization = (total_weight / specs["capacity"]) * 100
            on_time_delivery = total_distance / specs["max_distance"] <= 0.9
            
            trip_details = {
                "Trip_ID": f"T{trip_id}",
                "Sequence": [s["id"] for s in shipments],
                "Vehicle_Type": vehicle_type,
                "MST_DIST": total_distance,
                "TRIP_TIME": trip_time,
                "CAPACITY_UTI": capacity_utilization,
                "ON_TIME_DELIVERY": on_time_delivery,
            }
            
            print(trip_details)
            trips.append(trip_details)
            vehicles[vehicle_type]["count"] -= 1
            trip_id += 1

            plot_route_map(optimized_route, trip_details)
            break

    return trips

def main():
    """Main execution function"""
    print("Welcome to the Delivery Route Optimization System")
    
    # Get shipment data
    shipments = get_shipments_from_user()
    
    # Process and optimize routes
    print("\nProcessing and optimizing routes...")
    trips = plan_trips(shipments)

    # Display results
    print("\nOptimized Delivery Trips:")
    for trip in trips:
        print("\nTrip Details:")
        for key, value in trip.items():
            print(f"{key}: {value}")

if __name__ == "__main__":
    main()


Road network loaded successfully!
Welcome to the Delivery Route Optimization System

Processing and optimizing routes...
{'Trip_ID': 'T1', 'Sequence': ['S1', 'S2', 'S3'], 'Vehicle_Type': '4W', 'MST_DIST': 18.26248778980873, 'TRIP_TIME': 121.31243894904365, 'CAPACITY_UTI': 68.0, 'ON_TIME_DELIVERY': True}
Route map saved as Trip_T1_Route_Map.html

Optimized Delivery Trips:

Trip Details:
Trip_ID: T1
Sequence: ['S1', 'S2', 'S3']
Vehicle_Type: 4W
MST_DIST: 18.26248778980873
TRIP_TIME: 121.31243894904365
CAPACITY_UTI: 68.0
ON_TIME_DELIVERY: True
